Пришло время для дл, пока я копался с ml, мои товарищи по команде уже закочили 2 модели(табнет и простой табнет и простой перцептрон), при условии что нам нужно осмысленное отличие модели я отановлбсь на свёртке.
Пред обработаем данные так каки в ML и закодируем целевую переменную лейбл инкодером как это делали в MLP

In [15]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import warnings
from sklearn.metrics import accuracy_score, f1_score
import time
warnings.filterwarnings('ignore')
import pandas as pd
from sklearn.model_selection import train_test_split

train = pd.read_csv('train.csv')
split_idx = int(0.8 * len(train))
test = train.iloc[split_idx:]
train = train.iloc[:split_idx]

X_train = train.drop('Fertilizer Name', axis=1)
y_train = train['Fertilizer Name']

X_test = test.drop('Fertilizer Name', axis=1)
y_test = test['Fertilizer Name']


from sklearn.preprocessing import OneHotEncoder
cat_cols = ['Soil Type', 'Crop Type']
encoder = OneHotEncoder(handle_unknown='ignore')
X_train_encoded = encoder.fit_transform(X_train[cat_cols]).toarray()
feature_names = encoder.get_feature_names_out(cat_cols)
X_test_encoded = encoder.transform(X_test[cat_cols]).toarray()

from sklearn.preprocessing import StandardScaler
num_cols = ['Temparature', 'Humidity', 'Moisture', 'Nitrogen', 'Potassium', 'Phosphorous']
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train[num_cols])
X_test_scaled = scaler.transform(X_test[num_cols])

X_train_processed = np.hstack([X_train_scaled, X_train_encoded])
X_test_processed = np.hstack([X_test_scaled, X_test_encoded])
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

In [18]:
!nvidia-smi --query-gpu=gpu_name,memory.total,memory.free,memory.used --format=csv

/bin/bash: line 1: nvidia-smi: command not found


In [22]:

import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm import tqdm


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


X_train_tensor = torch.tensor(X_train_processed, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train_encoded, dtype=torch.long).to(device)
X_test_tensor = torch.tensor(X_test_processed, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test_encoded, dtype=torch.long).to(device)


train_size = int(0.8 * len(X_train_tensor))
val_size = len(X_train_tensor) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(
    TensorDataset(X_train_tensor, y_train_tensor),
    [train_size, val_size]
)

batch_size = 128 if device.type == 'cuda' else 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(TensorDataset(X_test_tensor, y_test_tensor), batch_size=batch_size)


def calculate_accuracy(loader, model):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

def evaluate_model(loader, model):
    model.eval()
    all_labels = []
    all_preds = []
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(predicted.cpu().numpy())
    return all_labels, all_preds

input_size = X_train_processed.shape[1]
num_classes = len(np.unique(y_train_encoded))
cnn_model = CNN1D_FertilizerClassifier(input_size, num_classes).to(device)

criterion = nn.CrossEntropyLoss()
cnn_optimizer = optim.Adam(cnn_model.parameters(), lr=0.001, weight_decay=1e-4)
cnn_scheduler = ReduceLROnPlateau(cnn_optimizer, mode='max', factor=0.5, patience=2, verbose=True)


print(f"\nОбучение CNN модели на {device} (7 эпох)...")
best_val_accuracy_cnn = 0

for epoch in range(1):
    cnn_model.train()
    running_loss = 0.0

    progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/7', leave=True)
    for inputs, labels in progress_bar:
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = cnn_model(inputs)
        loss = criterion(outputs, labels)

        cnn_optimizer.zero_grad()
        loss.backward()
        cnn_optimizer.step()

        running_loss += loss.item()
        progress_bar.set_postfix(loss=running_loss/(progress_bar.n+1))


    train_accuracy = calculate_accuracy(train_loader, cnn_model)
    val_accuracy = calculate_accuracy(val_loader, cnn_model)

    print(f"Epoch [{epoch+1}/7], Loss: {running_loss/len(train_loader):.4f}, "
          f"Train Acc: {train_accuracy:.4f}, Val Acc: {val_accuracy:.4f}")


    cnn_scheduler.step(val_accuracy)

    if val_accuracy > best_val_accuracy_cnn:
        best_val_accuracy_cnn = val_accuracy
        torch.save(cnn_model.state_dict(), 'best_cnn_model.pth')


cnn_model.load_state_dict(torch.load('best_cnn_model.pth'))

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

test_labels, test_preds = evaluate_model(test_loader, cnn_model)

accuracy = accuracy_score(test_labels, test_preds)
precision = precision_score(test_labels, test_preds, average='weighted')
recall = recall_score(test_labels, test_preds, average='weighted')
f1 = f1_score(test_labels, test_preds, average='weighted')

print("\nМетрики CNN модели после 7 эпох обучения:")
print(f"Accuracy:  {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1-score:  {f1:.4f}")


Обучение CNN модели на cpu (7 эпох)...


Epoch 1/7: 100%|██████████| 7500/7500 [08:32<00:00, 14.64it/s, loss=1.94]


Epoch [1/7], Loss: 1.9430, Train Acc: 0.1502, Val Acc: 0.1478

Метрики CNN модели после 7 эпох обучения:
Accuracy:  0.1483
Precision: 0.0866
Recall:    0.1483
F1-score:  0.0683
